In [2]:
import os
import polars as pl


In [83]:
df_medicamentos = pl.read_parquet('./data/medicamentos_preprocesados.parquet')

In [94]:
def target_encoding_ranking_polars(df, columna):
    try:
        # 1. Calcular frecuencias y ranking (igual que antes)
        freq_ranking = (
            df
            .group_by(columna)
            .agg(pl.len().alias('count'))
            .sort('count', descending=True)
            .with_row_index('ranking', offset=1)
            .with_columns([
                (pl.col('ranking') + (pl.col('count') / 10000.0))
                .alias('encoding_value')
            ])
        )
        
        # 2. Crear diccionario de mapeo
        encoding_map = dict(
            zip(
                freq_ranking[columna].to_list(),
                freq_ranking['encoding_value'].to_list()
            )
        )
        
        return encoding_map
        
    except Exception as e:
        print(f"Error en target_encoding_ranking_polars: {e}")
        return {}

def one_hot_encoding_polars(df, columna):
    try:
        # Obtener valores únicos de la columna
        valores_unicos = df[columna].unique().to_list()
        
        # Crear columnas one-hot
        expresiones_onehot = []
        for valor in valores_unicos:
            # Limpiar nombre de columna (remover espacios y caracteres especiales)
            nombre_columna = f"{columna}_{valor}".replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_")
            
            # Crear expresión binaria
            expresion = (pl.col(columna) == valor).cast(pl.Int8).alias(nombre_columna)
            expresiones_onehot.append(expresion)
        
        # Aplicar todas las expresiones one-hot Y eliminar columna original
        df_onehot = df.with_columns(expresiones_onehot).drop(columna)
        
        return df_onehot
        
    except Exception as e:
        print(f"Error en one_hot_encoding_polars: {e}")
        return df

def normalizar_columna_polars(df, columna, metodo='minmax'):
    """
    Normaliza una columna numérica usando MinMax o Standard Scaler.
    
    Args:
        df (pl.DataFrame): DataFrame de Polars
        columna (str): Nombre de la columna numérica a normalizar
        metodo (str): 'minmax' o 'standard'
    
    Returns:
        pl.DataFrame: DataFrame con la columna normalizada (reemplaza la original)
        
    Ejemplos:
        MinMax: valores entre 0 y 1
        Standard: media=0, desviación=1
    """
    try:
        if metodo == 'minmax':
            # MinMax: (valor - min) / (max - min)
            df_normalizado = df.with_columns([
                (
                    (pl.col(columna) - pl.col(columna).min()) / 
                    (pl.col(columna).max() - pl.col(columna).min())
                ).alias(f"{columna}")
            ])
            
        elif metodo == 'standard':
            # Standard: (valor - media) / desviación
            df_normalizado = df.with_columns([
                (
                    (pl.col(columna) - pl.col(columna).mean()) / 
                    pl.col(columna).std()
                ).alias(f"{columna}")
            ])
            
        else:
            print(f"Método '{metodo}' no reconocido. Usa 'minmax' o 'standard'")
            return df
            
        return df_normalizado
        
    except Exception as e:
        print(f"Error en normalizar_columna_polars: {e}")
        return df

In [105]:

df_medicamentos_train = df_medicamentos.select([
 'ATC',                 # 2,419 únicos - Código farmacológico
 'VÍA ADMINISTRACIÓN',  # 51 únicos - Vìa de administración 
 'FORMA FARMACÉUTICA',  # 112 únicos - Presentación física
 'PRINCIPIO ACTIVO',    # 20,321 únicos - Ingrediente activo
 'ESTADO REGISTRO',     # 14 únicos - Estado regulatorio
 'ESTADO CUM',          # 2 únicos - Estado del CUM
 'MUESTRA MÉDICA',      # 2 únicos - Si es muestra
 
 'CANTIDAD CUM',        # Numérica - Cantidad presentación
 'UNIDAD',              # 13 únicos - U, ml, g   

 'CANTIDAD',            # Numérica - Dosis
 'UNIDAD MEDIDA',       # 180 únicos - mg, g, mcg
 'EXPEDIENTE CUM'       # 34,653 únicos - Expediente (parte de la familia principal del medicamento)
 ])

# Convert EXPEDIENTE CUM from int to str
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('EXPEDIENTE CUM').cast(pl.Utf8)
)

df_medicamentos_train.head(5)

ATC,VÍA ADMINISTRACIÓN,FORMA FARMACÉUTICA,PRINCIPIO ACTIVO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA,CANTIDAD CUM,UNIDAD,CANTIDAD,UNIDAD MEDIDA,EXPEDIENTE CUM
str,str,str,str,str,str,str,f64,str,f64,str,str
"""J05AR26""","""ORAL""","""TABLETA CUBIERTA CON PELICULA""","""DARUNAVIR ETANOLATO 867.6 MG. …","""Vigente""","""Activo""","""No""",30.0,"""U""",800.0,"""mg""","""20167292"""
"""A11AA01""","""ORAL""","""CÁPSULA BLANDA DE GELATINA CON…","""DL-ALFA TOCOFERIL ACETATO / VI…","""Vigente""","""Activo""","""No""",1.0,"""U""",10.0,"""mg""","""20015204"""
"""B05BA10""","""PARENTERAL""","""EMULSION INYECTABLE""","""L-SERINA""","""Vigente""","""Inactivo""","""No""",1.0,"""U""",6.5,"""g""","""20061748"""
"""B05BA10""","""INTRAVENOSA""","""EMULSION INYECTABLE""","""L-TREONINA""","""Vigente""","""Inactivo""","""Si""",1.0,"""U""",2.2,"""mg""","""20061746"""
"""N02BE51""","""ORAL""","""CAPSULA DURA""","""CAFEINA ANHIDRA""","""Temp. no comerc - Vigente""","""Activo""","""No""",50.0,"""U""",30.0,"""mg""","""20009806"""


In [106]:
atc_map = target_encoding_ranking_polars(df_medicamentos, 'ATC')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('ATC').replace(atc_map).cast(pl.Float64).alias('ATC')
)

df_medicamentos_train = one_hot_encoding_polars(df_medicamentos_train, 'VÍA ADMINISTRACIÓN')

df_medicamentos_train = one_hot_encoding_polars(df_medicamentos_train, 'FORMA FARMACÉUTICA')

principio_map = target_encoding_ranking_polars(df_medicamentos, 'PRINCIPIO ACTIVO')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('PRINCIPIO ACTIVO').replace(principio_map).cast(pl.Float64).alias('PRINCIPIO ACTIVO')
)

df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('ESTADO REGISTRO').map_elements(lambda x: 1 if x == "Vigente" else 0, return_dtype=pl.Int8),
    pl.col('ESTADO CUM').map_elements(lambda x: 1 if x == "Activo" else 0, return_dtype=pl.Int8),
    pl.col('MUESTRA MÉDICA').map_elements(lambda x: 1 if x == "No" else 0, return_dtype=pl.Int8),
)

df_medicamentos_train = normalizar_columna_polars(df_medicamentos_train, 'CANTIDAD CUM', 'standard')
df_medicamentos_train = normalizar_columna_polars(df_medicamentos_train, 'CANTIDAD', 'standard')

unidad_map = target_encoding_ranking_polars(df_medicamentos, 'UNIDAD')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('UNIDAD').replace(unidad_map).cast(pl.Float64).alias('UNIDAD')
)

unimed_map = target_encoding_ranking_polars(df_medicamentos, 'UNIDAD MEDIDA')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('UNIDAD MEDIDA').replace(unimed_map).cast(pl.Float64).alias('UNIDAD MEDIDA')
)

expediente_map = target_encoding_ranking_polars(df_medicamentos, 'EXPEDIENTE CUM')
df_medicamentos_train = df_medicamentos_train.with_columns(
    pl.col('EXPEDIENTE CUM').replace(expediente_map).cast(pl.Float64).alias('EXPEDIENTE CUM')
)

df_medicamentos_train.head(5)

ATC,PRINCIPIO ACTIVO,ESTADO REGISTRO,ESTADO CUM,MUESTRA MÉDICA,CANTIDAD CUM,UNIDAD,CANTIDAD,UNIDAD MEDIDA,EXPEDIENTE CUM,VÍA_ADMINISTRACIÓN_PARENTERAL,VÍA_ADMINISTRACIÓN_RAQUIDEA,VÍA_ADMINISTRACIÓN_INTRAOCULAR,VÍA_ADMINISTRACIÓN_OFTÁLMICA,VÍA_ADMINISTRACIÓN_BUCAL,VÍA_ADMINISTRACIÓN_INTRAVESICAL,VÍA_ADMINISTRACIÓN_INFILTRATIVA___LOCAL,VÍA_ADMINISTRACIÓN_INTRACAVERNOSA,VÍA_ADMINISTRACIÓN_HEMODIALISIS,VÍA_ADMINISTRACIÓN_TRANSDERMAL,VÍA_ADMINISTRACIÓN_INTRANASAL,VÍA_ADMINISTRACIÓN_INTRAPLEURAL,VÍA_ADMINISTRACIÓN_VAGINAL,VÍA_ADMINISTRACIÓN_URETRAL,VÍA_ADMINISTRACIÓN_INFUSIÓN_INTRAVENOSA,VÍA_ADMINISTRACIÓN_PERIARTICULAR,VÍA_ADMINISTRACIÓN_IRRIGACIÓN,VÍA_ADMINISTRACIÓN_INTRADERMAL,VÍA_ADMINISTRACIÓN_DENTAL,VÍA_ADMINISTRACIÓN_INTRAMUSCULAR,VÍA_ADMINISTRACIÓN_INHALACION,VÍA_ADMINISTRACIÓN_INSUFLACION,VÍA_ADMINISTRACIÓN_INTRAVENOSA,VÍA_ADMINISTRACIÓN_SIN_DATO,VÍA_ADMINISTRACIÓN_INTRA_ARTICULAR,VÍA_ADMINISTRACIÓN_INFILTRATIVA___EPIDURAL,VÍA_ADMINISTRACIÓN_INTRALESIONAL,…,FORMA_FARMACÉUTICA_SOLUCION_OFTALMICA,FORMA_FARMACÉUTICA_ESPONJA_MEDICADA,FORMA_FARMACÉUTICA_INYECTABLES,FORMA_FARMACÉUTICA_JALEA,FORMA_FARMACÉUTICA_MEDICAMENTOS_NO_ESTERILES_SOLIDOS___POLVOS_Y_GRANULADOS,FORMA_FARMACÉUTICA_SOLUCIÓN_CONCENTRADA_PARA_INFUSIÓN,FORMA_FARMACÉUTICA_SOLUCION_PARA_ADMINISTRAR_POR_VIA_RECTAL,FORMA_FARMACÉUTICA_EMULSION_CHAMPOO,FORMA_FARMACÉUTICA_TABLETA_RECUBIERTA,FORMA_FARMACÉUTICA_LOCION,FORMA_FARMACÉUTICA_UNGUENTO_TOPICO,FORMA_FARMACÉUTICA_TABLETA_CON_CUBIERTA_ENTERICAGRAGEA,FORMA_FARMACÉUTICA_ELIXIR,FORMA_FARMACÉUTICA_TRANSDERMICOS,FORMA_FARMACÉUTICA_CÁPSULA_BLANDA_DE_GELATINA_CON_CUBIERTA_ENTÉRICA,FORMA_FARMACÉUTICA_TABLETA_BUCAL,FORMA_FARMACÉUTICA_TABLETA_DE_LIBERACION_RETARDADA,FORMA_FARMACÉUTICA_EMULSION_ORAL,FORMA_FARMACÉUTICA_GEL_VAGINAL,FORMA_FARMACÉUTICA_COMPRIMIDO,FORMA_FARMACÉUTICA_CREMAS._GELES._UNGUENTOS_Y_PASTAS,FORMA_FARMACÉUTICA_TABLETA_DISPERSABLE,FORMA_FARMACÉUTICA_SUSPENSION_PARA_INHALACION,FORMA_FARMACÉUTICA_PASTA,FORMA_FARMACÉUTICA_SOLUCIONNASAL,FORMA_FARMACÉUTICA_SUPOSITORIO,FORMA_FARMACÉUTICA_SOLUCION_PARA_NEBULIZACION,FORMA_FARMACÉUTICA_LAMINAS_DISPERSABLES,FORMA_FARMACÉUTICA_UNGUENTO_OFTÁLMICO,FORMA_FARMACÉUTICA_TABLETA_DE_LIBERACIÓN_MODIFICADA,FORMA_FARMACÉUTICA_AEROSOLES,FORMA_FARMACÉUTICA_CAPSULAS_DURAS_Y_BLANDAS,FORMA_FARMACÉUTICA_TINTURA_ORAL,FORMA_FARMACÉUTICA_GOMAS,FORMA_FARMACÉUTICA_GRANULOS,FORMA_FARMACÉUTICA_SOLUCION_PARA_PRESERVACION_DE_ORGANOS_EN_TRANSPLANTES,FORMA_FARMACÉUTICA_SISTEMAS_DE_LIBERACION
f64,f64,i8,i8,i8,f64,f64,f64,f64,f64,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,…,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8,i8
1773.0006,13131.0002,1,1,1,-0.071174,22.4076,-0.00413,19.1932,15438.0004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9.2139,2329.0016,1,1,1,-0.091356,22.4076,-0.004135,19.1932,35.0224,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7.284,216.0152,1,0,1,-0.091356,22.4076,-0.004135,6.2562,28.0243,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7.284,224.015,1,0,0,-0.091356,22.4076,-0.004135,19.1932,27.0252,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2.8577,12.1041,0,1,1,-0.057256,22.4076,-0.004135,19.1932,742.004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [101]:
print(df_medicamentos_train.shape)
print(df_medicamentos_train.dtypes)

(248635, 173)
[Float64, Float64, Int8, Int8, Int8, Float64, Float64, Float64, Float64, Float64, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8, Int8